# 💡 这节课会带给你

1. 系统性维护、测试、监控一个 LLM 应用
2. 学习使用主流的工具完成上述工作

## 维护一个生产级的 LLM 应用，我们需要做什么？

1. 调试 Prompt
2. Prompt 版本管理
3. 测试/验证系统的相关指标
4. 数据集管理
5. 各种指标监控与统计：访问量、响应时长、Token费等等

### 针对以上需求，我们介绍三个生产级 LLM App 维护平台

1. **LangSmith**: LangChain 的官方平台，SaaS 服务，非开源；
2. **LangFuse**: 开源 + SaaS，LangSmith 平替，可集成 LangChain 也可直接对接 OpenAI API；
3. **Prompt Flow**：微软开发，开源 + Azure AI云服务，可集成 Semantic Kernel。

## 1、LangSmith

平台入口：https://www.langchain.com/langsmith

文档地址：https://python.langchain.com/docs/langsmith/walkthrough

将你的 LangChain 应用与 LangSmith 链接，需要：

1. 注册账号，并申请一个`LANGCHAIN_API_KEY`
2. 在环境变量中设置以下值

```shell
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_PROJECT=YOUR_PROJECT_NAME #自定义项目名称
export LANGCHAIN_ENDPOINT=https://api.smith.langchain.com #LangSmith的服务端点
export LANGCHAIN_API_KEY=LANGCHAIN_API_KEY # LangChain API Key
```

3. 程序中的调用将自动被记录

In [12]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="agi_demo_hello_world"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__a5a4ca7a021342748af8127bb805ba0a"

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# 定义语言模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

# 定义Prompt模板
prompt = PromptTemplate.from_template("Say hello to {input}!")

# 定义输出解析器
parser = StrOutputParser()

chain = (
    {"input":RunnablePassthrough()} 
    | prompt
    | llm
    | parser
)

chain.invoke("中国")

'Hello 中国!'

<img src="langsmith-example.png" width="800px">

### 1.1、基本功能

1. Traces
2. LLM Calls
3. Monitor
4. Playground

### 1.2、Dataset & Evaluator

在产品中使用一个 AI 功能，我们首先需要系统性测试它的能力指标

In [ ]:
!pip install wikipedia

In [18]:
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter

prompt_template = """
Answer user's question according to the context below. 
Be brief, answer in no more than 20 words.
CONTEXT_START
{context}
CONTEXT_END

USER QUESTION:
{input}
"""

# 检索 wikipedia
retriever = WikipediaRetriever(top_k_results=3)

def chain_constructor(retriever):
    # 定义语言模型
    llm = ChatOpenAI(
        model="gpt-3.5-turbo-16k",
        temperature=0,
    )
    
    # 定义Prompt模板
    prompt = PromptTemplate.from_template(
        prompt_template
    )
    
    # 定义输出解析器
    parser = StrOutputParser()

    response_generator = (
        #{"context":retriever,"input":RunnablePassthrough()}
        prompt 
        | llm 
        | StrOutputParser()
    )
    
    chain = (
        {
            "context": itemgetter("input") | retriever | (lambda docs: "\n".join([doc.page_content for doc in docs])),
            "input":  itemgetter("input")
        }
        | response_generator
    )

    return chain

第一步，我们需要准备一个数据集，包含输入与预期输出

In [8]:
import json

qa_pairs = []
with open('example_dataset.jsonl','r',encoding='utf-8') as fp:
    for line in fp:
        example = json.loads(line.strip())
        qa_pairs.append(example)

In [14]:
from langsmith import Client

client = Client()

dataset_name = "wiki_qa_dataset_demo_100"

dataset = client.create_dataset(
    dataset_name, #数据集名称
    description="一个数据集样例，从wiki_qa benchmark中抽取的100条问答对", #数据集描述
)

for example in qa_pairs:
    client.create_example(
        inputs={"input": example['question']}, outputs={"output": example['answer']}, dataset_id=dataset.id
    )

HTTPError: [Errno 400 Client Error: Bad Request for url: https://api.smith.langchain.com/datasets] {"detail":"Dataset with this name already exists."}

第二步，我们定义评估函数，用于数值化的评估模型的实际输出与预期输出之间的差距

In [ ]:
from langchain.evaluation import EvaluatorType
from langchain.smith import RunEvalConfig

evaluation_config = RunEvalConfig(
    # 评估器，可多选
    evaluators=[
        # 根据答案判断回复是否"Correct"
        EvaluatorType.QA,
    ],
    # 可追加自定评估标准
    custom_evaluators=[],
)

In [ ]:
from langchain.smith import (
    arun_on_dataset,
    run_on_dataset,
)
from uuid import uuid4

unique_id = uuid4().hex[0:8]

chain = chain_constructor(retriever)

chain_results = await arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"AGIClass_LangChain_WikiQA_Project-{unique_id}",
    tags=[
        "testing-agiclass-demo",
        "2023-11-30",
    ],  # 可选，自定义的标识
)

View the evaluation results for project 'AGIClass_LangChain_WikiQA_Project-d37fcdde' at:
https://smith.langchain.com/o/97b8262a-9ab9-4b43-afeb-21ea05a90ba7/projects/p/32d10247-25be-49e5-8d83-619e0ea36228?eval=true

View all tests for Dataset wiki_qa_dataset_demo_100 at:
https://smith.langchain.com/o/97b8262a-9ab9-4b43-afeb-21ea05a90ba7/datasets/e08eb251-fd32-4fe4-93bc-95fcd30c7167
[------------------------------------------------->] 99/100

### 1.3、自定义评估指标

In [16]:
from langchain.evaluation import StringEvaluator
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import re
from typing import Optional, Any

class BleuEvaluator(StringEvaluator):

    def __init__(self):
        pass

    @property
    def requires_input(self) -> bool:
        return False

    @property
    def requires_reference(self) -> bool:
        return True

    @property
    def evaluation_name(self) -> str:
        return "bleu_score"

    def _tokenize(self,sentence):
        # 正则表达式定义了要去除的标点符号
        return re.sub(r'[^\w\s]', '', sentence.lower()).split()
    
    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        bleu_score = sentence_bleu(
            [self._tokenize(reference)], 
            self._tokenize(prediction), 
            smoothing_function=SmoothingFunction().method3
        )
        return {"score": bleu_score}

In [17]:
from uuid import uuid4
from langchain.smith import (
    arun_on_dataset,
    run_on_dataset,
)

evaluation_config = RunEvalConfig(
    # 自定义的BLEU SCORE评估器
    custom_evaluators=[BleuEvaluator()],
)

unique_id = uuid4().hex[0:8]
chain = chain_constructor(retriever)

chain_results = await arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=evaluation_config,
    verbose=True,
    client=client,
    project_name=f"AGIClass_LangChain_Custom_Project-{unique_id}",
    tags=[
        "testing-agiclass-demo",
        "2023-11",
    ],  # 可选，自定义的标识
)

View the evaluation results for project 'AGIClass_LangChain_Custom_Project-2b31e247' at:
https://smith.langchain.com/o/97b8262a-9ab9-4b43-afeb-21ea05a90ba7/projects/p/9f795ab0-376c-48c4-b3ce-37cefef756dd?eval=true

View all tests for Dataset wiki_qa_dataset_demo_100 at:
https://smith.langchain.com/o/97b8262a-9ab9-4b43-afeb-21ea05a90ba7/datasets/e08eb251-fd32-4fe4-93bc-95fcd30c7167
[------------------------------------------------->] 100/100

**额外知识**: BLEU Score

传统NLP中，对机器翻译或文本生成类模型效果的自动评估的常用方法之一，原理如下：
  - 计算输出与参照句之间的 n-gram 准确率（n=1...4）
  - 对短输出做惩罚
  - 在整个测试集上平均下述值

$\mathrm{BLEU}_4=\min\left(1,\frac{output-length}{reference-length}\right)\left(\prod_{i=1}^4 precision_i\right)^{\frac{1}{4}}$

## 2、说说文本生成常用的评估方法

假设文本生成问题，我们有或没有参考答案 reference 时，

怎么评估模型生成的结果的优劣？

### 2.1、基于大模型本身做评估

https://docs.smith.langchain.com/evaluation/evaluator-implementations

1. 正确性（Correctness）：用 LLM 判断给定真实答案的前提下，模型生成的答案是否正确

```python
from langchain.chat_models import ChatAnthropic
from langchain.prompts.prompt import PromptTemplate

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
Respond with CORRECT or INCORRECT:
Grade:
"""

PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)
eval_llm = ChatAnthropic(temperature=0.0)
evaluation_config = RunEvalConfig(
    evaluators=[
        RunEvalConfig.QA(llm=eval_llm, prompt=PROMPT),
        RunEvalConfig.ContextQA(llm=eval_llm),
        RunEvalConfig.CoTQA(llm=eval_llm),
    ]
)
```

2. 符合标准（Criteria）：无参考答案时，判断输出是否符合特定标准

```python
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.Criteria({"creativity": "Is this submission creative, imaginative, or novel?"}),
        # We provide some simple default criteria like "conciseness" you can use as well
        RunEvalConfig.Criteria("conciseness"),
    ]
)
```

3. 有帮助（Helpfulness）：根据参考答案判断模型输出是否有帮助

```python
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        # You can define an arbitrary criterion as a key: value pair in the criteria dict
        RunEvalConfig.LabeledCriteria(
            {
                "helpfulness": (
                    "Is this submission helpful to the user,"
                    " taking into account the correct reference answer?"
                )
            }
        ),
    ]
)

```

<div class="alert alert-success">
<b>划重点：</b>此类方法，对于用于评估的 LLM 自身能力有要求。需根据具体情况选择使用。
</div>

### 2.2、一些经典 NLP 的评测方法

1. **编辑距离**：也叫莱文斯坦距离(Levenshtein),是针对二个字符串的差异程度的量化量测，量测方式是看至少需要多少次的处理才能将一个字符串变成另一个字符串。
   - 具体计算过程是一个动态规划算法：https://zhuanlan.zhihu.com/p/164599274
   - 衡量两个句子的相似度时，可以以词为单位计算
2. **BLEU Score**:
   - 计算输出与参照句之间的 n-gram 准确率（n=1...4）
   - 对短输出做惩罚
   - 在整个测试集上平均下述值
3. **Rouge Score**:
   - Rouge-N：将模型生成的结果和标准结果按 N-gram 拆分后，只计算召回率；
   - Rouge-L: 利用了最长公共子序列（Longest Common Sequence），计算：$P=\frac{LCS(c,r)}{len(c)}$, $R=\frac{LCS(c,r)}{len(r)}$, $F=\frac{(1+\beta^2)PR}{R+\beta^2P}$
   - 函数库：https://pypi.org/project/rouge-score/
   - 对比 BLEU 与 ROUGE：
     - BLEU 能评估流畅度，但指标偏向于较短的翻译结果（brevity penalty 没有想象中那么强）
     - ROUGE 不管流畅度，所以只适合深度学习的生成模型：结果都是流畅的前提下，ROUGE 反应参照句中多少内容被生成的句子包含（召回）
5. **METEOR**: 另一个从机器翻译领域借鉴的指标。与 BLEU 相比，METEOR 考虑了更多的因素，如同义词匹配、词干匹配、词序等，因此它通常被认为是一个更全面的评价指标。
   - 对语言学和语义词表有依赖，所以对语言依赖强。

<div class="alert alert-success">
<b>划重点：</b>此类方法常用于对文本生成模型的自动化评估。实际使用中，我们通常更关注相对变化而不是绝对值（调优过程中指标是不是在变好）。
</div>

## 3、LangFuse

功能与 LangSmith 基本重合，开源，支持 LangChain 集成或原生 OpenAI API 集成

<img src="langfuse.png" width="600px">

1. 通过官方云服务使用：
   - 注册: cloud.langfuse.com
   - 创建 API Key

```sh
LANGFUSE_SECRET_KEY="sk-lf-..."
LANGFUSE_PUBLIC_KEY="pk-lf-..."
```

2. 通过 Docker 本地部署

```sh
# Clone repository
git clone https://github.com/langfuse/langfuse.git
cd langfuse
 
# Run server and db
docker compose up -d
```

In [ ]:
!pip install langfuse

### 3.1、替换 OpenAI 客户端

In [1]:
from datetime import datetime
from langfuse.openai import openai
import os

completion = openai.chat.completions.create(
  name="test-chat",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "你是个测试版机器人。"},
      {"role": "user", "content": "对我说'Hello, World!'"}],
  temperature=0,
)

print(completion.choices[0].message.content)

Hello, World!


### 3.2、通过 LangChain 的回调集成

In [10]:
from langfuse.callback import CallbackHandler

handler = CallbackHandler(
    os.getenv("LANGFUSE_PUBLIC_KEY"), 
    os.getenv("LANGFUSE_SECRET_KEY")
)

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

from langchain.chat_models import ErnieBotChat
from langchain.schema import HumanMessage
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

baidu_model = ErnieBotChat()

prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("Say hello to {input}!") 
])


# 定义输出解析器
parser = StrOutputParser()

chain = (
    {"input":RunnablePassthrough()} 
    | prompt
    | baidu_model
    | parser
)

In [13]:
chain.invoke("文心", config={"callbacks":[handler]})

ERROR:langfuse:'model_name'
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/langfuse/callback.py", line 522, in __on_llm_action
    model_name = kwargs["invocation_params"]["model_name"]
                 ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
KeyError: 'model_name'
ERROR:langfuse:run not found
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/langfuse/callback.py", line 593, in on_llm_end
    raise Exception("run not found")
Exception: run not found


'文心 (wenhunjing) 是个什么产品/系统呢？它有着什么样的特点和优势？您可以尝试与我分享更多关于文心系统或者相关方面的信息，我很乐意尝试帮助您解答关于文心的各种问题。'

### 3.3、数据集与测试

1. 创建数据集

In [5]:
import json

qa_pairs = []
with open('example_dataset.jsonl','r',encoding='utf-8') as fp:
    for line in fp:
        example = json.loads(line.strip())
        qa_pairs.append(example)

In [6]:
from langfuse import Langfuse
from langfuse.model import CreateDatasetRequest, CreateDatasetItemRequest
 
# init
langfuse = Langfuse()

langfuse.create_dataset(CreateDatasetRequest(name="wiki_qa-20"));

for item in qa_pairs[:20]:
  langfuse.create_dataset_item(
    CreateDatasetItemRequest(
        dataset_name="wiki_qa-20",
        # any python object or value
        input=item["question"],
        # any python object or value, optional
        expected_output=item["answer"]
    )
)

2. 定义评估函数

In [7]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import re

def bleu_score(output, expected_output):
    def _tokenize(sentence):
        # 正则表达式定义了要去除的标点符号
        return re.sub(r'[^\w\s]', '', sentence.lower()).split()
    
    return sentence_bleu(
        [_tokenize(expected_output)], 
        _tokenize(output), 
        smoothing_function=SmoothingFunction().method3
    )

3. 定义 Chain

In [8]:
from langchain.retrievers import WikipediaRetriever
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

prompt_template = """
Answer user's question according to the context below. 
Be brief, answer in no more than 20 words.
CONTEXT_START
{context}
CONTEXT_END

USER QUESTION:
{input}
"""


# 定义语言模型
llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0,
)

# 定义Prompt模板
prompt = PromptTemplate.from_template(
    prompt_template
)

# 检索 wikipedia
retriever = WikipediaRetriever(top_k_results=1)


# 定义输出解析器
parser = StrOutputParser()

wiki_qa_chain = (
    {
        "context": retriever, 
        "input": RunnablePassthrough()
    } 
    | prompt
    | llm
    | parser
)


4. 运行测试

In [9]:
from langfuse.client import CreateScore
from tqdm import tqdm

dataset = langfuse.get_dataset("wiki_qa-20")

for item in tqdm(dataset.items):
    handler = item.get_langchain_handler(run_name="test_wiki_qa-20")

    output = wiki_qa_chain.invoke(item.input, config={"callbacks":[handler]})
    
    handler.rootSpan.score(CreateScore(
      name="bleu_score",
      value=bleu_score(output, item.expected_output)
    ))

  0%|          | 0/20 [00:00<?, ?it/s]ERROR:langchain.callbacks.tracers.langchain:Authentication failed for https://api.smith.langchain.com/runs/64d5422b-c40f-4c93-8b8b-a98ed0a9e266. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/64d5422b-c40f-4c93-8b8b-a98ed0a9e266', '{"detail":"Invalid auth"}')
ERROR:langchain.callbacks.tracers.langchain:Authentication failed for https://api.smith.langchain.com/runs. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs', '{"detail":"Invalid auth"}')
100%|██████████| 20/20 [01:06<00:00,  3.31s/it]


### 3.4、基于 LLM 的测试方法

LangFuse 集成了一些原生的基于 LLM 的自动测试标准。

具体参考：https://langfuse.com/docs/scores/model-based-evals

## 4、Prompt Flow

项目地址 https://github.com/microsoft/promptflow

### 4.1、安装

```sh
pip install promptflow promptflow-tools
```

### 4.2、命令行运行

```sh
pf flow init --flow ./my_chatbot --type chat
```

### 4.3、VSCode 插件

https://marketplace.visualstudio.com/items?itemName=prompt-flow.prompt-flow

<img src="vsc.png" width="800px">

### 4.4、与 Semantic Kernel 结合使用

<演示>

Azure云服务：https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/get-started-prompt-flow?view=azureml-api-2

## 总结

管理一个 LLM 应用的全生命周期，需要用到以下工具：

1. 调试 Prompt 的 Playground
2. 测试/验证系统的相关指标
3. 数据集管理
4. 各种指标监控与统计：访问量、响应时长、Token费等等

根据自己的技术栈，选择：

1. LangSmith: LangChain 的原始管理平台
2. LangFuse：开源平台，支持 LangChain 和原生 OpenAI API
3. Prompt Flow：开源平台，支持 Semantic Kernel

## 作业

选择一个工具平台，对自己之前开发的系统或模型做批量测试